In [19]:
import pandas as pd
df_pre = pd.read_csv('updated_dataframe.csv')
df_pre = df_pre.drop(columns=['Image URL'])
pd.options.display.max_columns = None
display(df_pre.head())

,ID,Style,agreeableness,anger,anticipation,arrogance,disagreeableness,disgust,fear,gratitude,happiness,humility,love,optimism,pessimism,regret,sadness,shame,shyness,surprise,trust,neutral,Image Path
0,58c6237dedc2c9c7dc0de1ae,Modern Art,0.060,0.012,0.071,0.024,0.012,0.000,0.012,0.119,0.726,0.369,0.250,0.274,0.012,0.000,0.131,0.000,0.024,0.024,0.250,0.0,resized_images\1_resized.png
1,577280dfedc2cb3880f28e76,Modern Art,0.000,0.000,0.100,0.000,0.000,0.100,0.300,0.000,0.100,0.100,0.000,0.200,0.200,0.100,0.200,0.000,0.000,0.500,0.000,0.0,resized_images\2_resized.png
2,57727f2dedc2cb3880ed5fa9,Modern Art,0.000,0.000,0.200,0.000,0.000,0.000,0.000,0.100,0.500,0.300,0.000,0.300,0.000,0.000,0.000,0.000,0.000,0.100,0.200,0.0,resized_images\3_resized.png
3,58d1240cedc2c94f900fc610,Modern Art,0.000,0.000,0.091,0.000,0.000,0.091,0.000,0.000,0.091,0.091,0.000,0.000,0.182,0.000,0.364,0.000,0.000,0.273,0.000,0.0,resized_images\4_resized.png
4,57727de7edc2cb3880e91f26,Post Renaissance Art,0.077,0.077,0.077,0.231,0.308,0.308,0.308,0.231,0.154,0.077,0.077,0.231,0.231,0.077,0.231,0.154,0.077,0.154,0.231,0.0,resized_images\5_resized.png


In [18]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split



# Data preprocessing function for images
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (600, 600))
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

# Create an ImageDataGenerator for real images
real_image_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_image,
    rescale=1./127.5 - 1  # Scale to the range [-1, 1]
)

# Split the dataset into training and testing
train_df, test_df = train_test_split(df_pre, test_size=0.2, random_state=42)

# Define the generator model
def build_generator(latent_dim, num_classes):
    noise = layers.Input(shape=(latent_dim,))
    label = layers.Input(shape=(num_classes,))
    
    x = layers.Concatenate()([noise, label])
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Reshape((4, 4, 16))(x)
    
    # Upsample to (600, 600, 3)
    x = layers.Conv2DTranspose(256, (4, 4), strides=(4, 4), padding='same', activation='relu')(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(3, (3, 3), padding='same', activation='tanh')(x)
    
    generated_image = layers.Reshape((600, 600, 3))(x)
    
    return keras.Model([noise, label], generated_image)


# Define the discriminator model
def build_generator(latent_dim, num_classes):
    noise = layers.Input(shape=(latent_dim,))
    label = layers.Input(shape=(num_classes,))
    
    x = layers.Concatenate()([noise, label])
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Reshape((4, 4, 16))(x)
    x = layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', activation='relu')(x)
    
    # Add ZeroPadding2D to adjust the dimensions
    x = layers.ZeroPadding2D(padding=((1, 0), (0, 0)))(x)
    
    generated_image = layers.Conv2D(3, (4, 4), padding='same', activation='tanh')(x)
    
    return keras.Model([noise, label], generated_image)





# Build the CGAN
latent_dim = 100
img_shape = (600, 600, 3)
num_classes = len(df_pre.columns) - 1

discriminator = build_discriminator(img_shape, num_classes)
discriminator.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

generator = build_generator(latent_dim, num_classes)

discriminator.trainable = False

noise = layers.Input(shape=(latent_dim,))
label = layers.Input(shape=(num_classes,))
fake_image = generator([noise, label])
validity = discriminator([fake_image, label])

cgan = keras.Model([noise, label], validity)
cgan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Training loop
batch_size = 64
epochs = 10000

for epoch in range(epochs):
    for _ in range(train_df.shape[0] // batch_size):
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        labels = train_df.drop(columns='Image Path').sample(batch_size, axis=0).to_numpy()

        # Generate fake images using the generator
        fake_images = generator.predict([noise, labels])

        # Load a batch of real images and labels
        real_image_paths = train_df['Image Path'].sample(batch_size, replace=True).tolist()
        real_images = []

        for image_path in real_image_paths:
            image = Image.open(image_path)
            image = image.resize((600, 600))
            image = np.array(image)  # Convert to NumPy array
            image = (image.astype(np.float32) - 127.5) / 127.5  # Scale to range [-1, 1]
            real_images.append(image)

        real_images = np.array(real_images)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch([real_images, labels], np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch([fake_images, labels], np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = cgan.train_on_batch([noise, labels], np.ones((batch_size, 1)))

        print(f"Epoch {epoch}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss}")



ValueError: Exception encountered when calling layer "model_7" (type Functional).

Input 0 of layer "dense_18" is incompatible with the layer: expected axis -1 of input shape to have value 2880022, but received input with shape (None, 2582)

Call arguments received by layer "model_7" (type Functional):
  • inputs=['tf.Tensor(shape=(None, 17, 16, 3), dtype=float32)', 'tf.Tensor(shape=(None, 22), dtype=float32)']
  • training=None
  • mask=None

In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from PIL import Image

# Split the dataset into training and testing
train_df, test_df = train_test_split(df_pre, test_size=0.2, random_state=42)

# Define the generator model
def build_generator(latent_dim, num_classes):
    noise = layers.Input(shape=(latent_dim,))
    label = layers.Input(shape=(num_classes,))
    
    x = layers.Concatenate()([noise, label])
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Reshape((4, 4, 16))(x)
    
    # Upsample to (600, 600, 3)
    x = layers.Conv2DTranspose(256, (4, 4), strides=(4, 4), padding='same', activation='relu')(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    
    # Add ZeroPadding2D to adjust the dimensions
    x = layers.ZeroPadding2D(padding=((1, 0), (0, 0)))(x)
    
    generated_image = layers.Conv2D(3, (3, 3), padding='same', activation='tanh')(x)
    
    return keras.Model([noise, label], generated_image)

# Define the discriminator model
def build_discriminator(img_shape, num_classes):
    image = layers.Input(shape=img_shape)
    label = layers.Input(shape=(num_classes,))
    
    # Embed the label information
    label_embedding = layers.Embedding(num_classes, img_shape[0] * img_shape[1])(label)
    label_embedding = layers.Reshape((img_shape[0], img_shape[1], 1))(label_embedding)
    
    x = layers.Concatenate()([image, label_embedding])
    
    x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.2)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    return keras.Model([image, label], x)

# Build the CGAN
latent_dim = 100
img_shape = (600, 600, 3)
num_classes = len(df_pre.columns) - 1

discriminator = build_discriminator(img_shape, num_classes)
discriminator.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

generator = build_generator(latent_dim, num_classes)

discriminator.trainable = False

noise = layers.Input(shape=(latent_dim,))
label = layers.Input(shape=(num_classes,))
fake_image = generator([noise, label])
validity = discriminator([fake_image, label])

cgan = keras.Model([noise, label], validity)
cgan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Training loop (adjust this according to your data and needs)
batch_size = 64
epochs = 10000

for epoch in range(epochs):
    for _ in range(train_df.shape[0] // batch_size):
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        labels = train_df.drop(columns='Image Path').sample(batch_size, axis=0).to_numpy()
        
       # Generate fake images using the generator
        fake_images = generator.predict([noise, labels])

        # Load a batch of real images and labels
        real_image_paths = train_df['Image Path'].sample(batch_size, replace=True).tolist()
        real_images = []

        for image_path in real_image_paths:
            image = Image.open(image_path)
            image = image.resize((600, 600))
            image = np.array(image)  # Convert to NumPy array
            image = (image.astype(np.float32) - 127.5) / 127.5  # Scale to range [-1, 1]
            real_images.append(image)

        real_images = np.array(real_images)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch([real_images, labels], np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch([fake_images, labels], np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = cgan.train_on_batch([noise, labels], np.ones((batch_size, 1)))

        print(f"Epoch {epoch}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss}")



ValueError: Exception encountered when calling layer "reshape_15" (type Reshape).

total size of new array must be unchanged, input_shape = [22, 360000], output_shape = [600, 600, 1]

Call arguments received by layer "reshape_15" (type Reshape):
  • inputs=tf.Tensor(shape=(None, 22, 360000), dtype=float32)